<center><img src="./image/CLBLOGO.jpg" alt="创乐博" style="width: 300px;"/></center>

# 7.机器人视觉巡线

@－－－－湖南创乐博智能科技有限公司－－－－<br>
@  文件名：7.机器人视觉巡线.py <br>
@  版本：V2.0 <br>
@  author: zhulin<br>
@  说明：机器人视觉巡线
机器人基本运动！<br>

## 1.载入必要的库

In [1]:
import numpy as np
import cv2
from LOBOROBOT import LOBOROBOT # 载入机器人库
from gpiozero import Button
from gpiozero import LED
import time 
import threading # 线程
import ctypes
import inspect

## 2.bgr转jpeg

In [2]:
def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])

## 线程结束代码

In [3]:
def _async_raise(tid, exctype):
    tid = ctypes.c_long(tid)
    if not inspect.isclass(exctype):
        exctype = type(exctype)
    res = ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, ctypes.py_object(exctype))
    if res == 0:
        raise ValueError("invalid thread id")
    elif res != 1:
        ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, None)
        raise SystemError("PyThreadState_SetAsyncExc failed")
        
def stop_thread(thread):
    _async_raise(thread.ident, SystemExit)

In [4]:
# 按键及LED初始化
Btn  = Button(19,pull_up=True)   # 按键端口
Gpin    = LED(5)
Rpin    = LED(6)

#初始化舵机
clbrobot = LOBOROBOT()  # 实例化机器人对象
clbrobot.t_stop(0)  # 停止

## 控制舵机的角度

In [5]:
# Configure min and max servo pulse lengths
servo_min = 150  # Min pulse length out of 4096
servo_max = 600  # Max pulse length out of 4096

# 频率设置为50hz，适用于舵机系统。
clbrobot.set_servo_angle(10,90)  # 底座舵机 90 
clbrobot.set_servo_angle(9,50)  # 顶部舵机 145
time.sleep(0.5)


In [6]:
# 按键标志位
keyflag = 0

# 按键控制函数
def keysacn():
    global keyflag
    print('*****************************************') 
    print('* makerobo Button Pressed!*') 
    print('*****************************************')
    Rpin.on()  # 打开红色LED
    Gpin.off() # 关闭绿色LED
    keyflag = 1   # 按键标志位置1

def released():
    print("button was released")
    Rpin.off()   # 关闭红色LED
    Gpin.on()    # 打开绿色LED

# 按键中断函数
Btn.when_pressed = keysacn
Btn.when_released = released

In [7]:
# 载入显示库
import ipywidgets.widgets as widgets
from IPython.display import display
import libcamera
from picamera2 import Picamera2

image = widgets.Image(format='jpeg', width=160, height=120)
display(image)

def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])

Image(value=b'', format='jpeg', height='120', width='160')

In [8]:
picamera = Picamera2()
config = picamera.create_preview_configuration(main={"format": 'RGB888', "size": (160, 120)},
                                               raw={"format": "SRGGB12", "size": (1920, 1080)})
config["transform"] = libcamera.Transform(hflip=0, vflip=1)
picamera.configure(config)
picamera.start()

[0:01:22.929982464] [5940]  INFO Camera camera_manager.cpp:284 libcamera v0.1.0+118-563cd78e
[0:01:22.943012659] [7812]  INFO RPI pisp.cpp:653 libpisp version v1.0.2 fa44a258644a 22-11-2023 (21:59:22)
[0:01:22.957081759] [7812]  INFO RPI pisp.cpp:1112 Registered camera /base/axi/pcie@120000/rp1/i2c@80000/ov5647@36 to CFE device /dev/media2 and ISP device /dev/media1 using PiSP variant BCM2712_C0
[0:01:22.959597358] [5940]  INFO Camera camera_manager.cpp:284 libcamera v0.1.0+118-563cd78e
[0:01:22.970767912] [7817]  INFO RPI pisp.cpp:653 libpisp version v1.0.2 fa44a258644a 22-11-2023 (21:59:22)
[0:01:22.984059306] [7817]  INFO RPI pisp.cpp:1112 Registered camera /base/axi/pcie@120000/rp1/i2c@80000/ov5647@36 to CFE device /dev/media2 and ISP device /dev/media1 using PiSP variant BCM2712_C0
[0:01:22.988750240] [5940]  WARN V4L2 v4l2_pixelformat.cpp:338 Unsupported V4L2 pixel format Y16 
[0:01:22.988819382] [5940]  WARN V4L2 v4l2_pixelformat.cpp:338 Unsupported V4L2 pixel format RGB6
[0:01:

In [9]:
def Video_display():
    while 1:
        if keyflag == 1:
            # Capture the frames
            frame = picamera.capture_array()
            frame=cv2.flip(frame, 1) 
            # Crop the image
            crop_img =frame[60:120, 0:160]
            # Convert to grayscale
            gray = cv2.cvtColor(crop_img, cv2.COLOR_BGR2GRAY)
            # Gaussian blur
            blur = cv2.GaussianBlur(gray,(5,5),0)
            # Color thresholding
            ret,thresh1 = cv2.threshold(blur,60,255,cv2.THRESH_BINARY_INV)
            # Erode and dilate to remove accidental line detections
            mask = cv2.erode(thresh1, None, iterations=2)
            mask = cv2.dilate(mask, None, iterations=2)
            
           # Find the contours of the frame
            contours,hierarchy = cv2.findContours(mask.copy(), 1, cv2.CHAIN_APPROX_NONE)
            # Find the biggest contour (if detected)
            if len(contours) > 0:
                c = max(contours, key=cv2.contourArea)
                M = cv2.moments(c)
                
                cx = int(M['m10']/M['m00'])
                cy = int(M['m01']/M['m00'])
                
                cv2.line(crop_img,(cx,0),(cx,720),(255,0,0),1)
                cv2.line(crop_img,(0,cy),(1280,cy),(255,0,0),1)
                
                cv2.drawContours(crop_img, contours, -1, (0,255,0), 1)
                
                if cx >= 120:
                    clbrobot.turnRight(30,0)                 
                    print("Turn Left!")
                if cx < 120 and cx > 50:
                    clbrobot.t_up(50,0)
                    print("On Track!")
                if cx <= 50:
                    clbrobot.turnLeft(30,0)   
                    print("Turn Right")
            else:
                print("I don't see the line")   
                
            image.value = bgr8_to_jpeg(crop_img)

## 开启线程

In [10]:
t = threading.Thread(target=Video_display)
t.setDaemon(True)
t.start()

/tmp/ipykernel_5940/3481196328.py:2: DeprecationWarning: setDaemon() is deprecated, set the daemon attribute instead
  t.setDaemon(True)


button was released
*****************************************
* makerobo Button Pressed!*
*****************************************
Turn Left!
Turn Left!
Turn Left!
Turn Left!
Turn Left!
Turn Left!
Turn Left!
Turn Left!
Turn Left!
Turn Left!
Turn Left!
Turn Left!
Turn Left!
Turn Left!
Turn Left!
Turn Left!
Turn Left!
Turn Left!
Turn Left!
Turn Left!
Turn Left!
Turn Left!
Turn Left!
Turn Left!
Turn Left!
Turn Left!
Turn Left!
Turn Left!
Turn Left!
Turn Left!
Turn Left!
Turn Left!
Turn Left!
Turn Left!
Turn Left!
Turn Left!
Turn Left!
Turn Left!
Turn Left!
Turn Left!
Turn Left!
Turn Left!
Turn Left!
Turn Left!
Turn Left!
Turn Left!
Turn Left!
Turn Left!
Turn Left!
Turn Left!
Turn Left!
Turn Left!
Turn Left!
Turn Left!
Turn Left!
Turn Left!
Turn Left!
Turn Left!
Turn Left!
Turn Left!
Turn Left!
Turn Left!
Turn Left!
Turn Left!
Turn Left!
Turn Left!
Turn Left!
Turn Left!
Turn Left!
Turn Left!
Turn Left!
Turn Left!
Turn Left!
Turn Left!
Turn Left!
Turn Left!
Turn Left!
Turn Left!
Turn Left!

## 结束线程

In [12]:
stop_thread(t)

Turn Left!
Turn Left!


## 关闭电机

In [13]:
clbrobot.t_stop(0)       